In [93]:
# pip install pyodbc

In [15]:
import pandas as pd
import pyodbc
import os

In [16]:
# Define the parameters for connect to SQL Server database
driver_name = "ODBC Driver 17 for SQL Server"
server_name = 'TIT-20230930FXD\\SQLEXPRESS'
database_name = 'brazilian_ecommerce' # database_name = 'master'
trusted_connection = 'yes'

# Define connection string
conn_str = f'DRIVER={driver_name};SERVER={server_name};DATABASE={database_name};Trusted_Connection={trusted_connection}'

In [17]:
# อ่านข้อมูลจากไฟล์ CSV โดยใช้ pandas
df = pd.read_csv(r'data\\olist_order_items_dataset.csv')
df.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [18]:
# Change data type about date to datetime
for col_date in list(df.columns):
    if col_date[-4:] == "date" or \
        col_date[-9:] == "timestamp" or \
        col_date[-2:] == "at":
        df[col_date] = pd.to_datetime(df[col_date])
    else:
        continue

In [19]:
df.drop_duplicates(inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [21]:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [24]:
# sql_cmd = """
#     CREATE TABLE order_items(    
#         order_id VARCHAR(255),
#         order_item_id INT,
#         product_id VARCHAR(255),
#         seller_id VARCHAR(255),
#         shipping_limit_date DATETIME,
#         price REAL,
#         freight_value REAL
#     );
# """

# with pyodbc.connect(conn_str) as con:
#     con.execute(sql_cmd)

In [25]:
# กำหนดคำสั่ง SQL เพื่อ truncate ตาราง
truncate_query = '''
    TRUNCATE TABLE order_items
'''

# Execute คำสั่ง SQL เพื่อ truncate ตาราง
with pyodbc.connect(conn_str).cursor() as cursor:
    cursor.execute(truncate_query)
    
pyodbc.connect(conn_str).commit()

In [29]:
with pyodbc.connect(conn_str) as con:
    cursor = con.cursor()
    for index, row in df.iterrows():
        insert_query = """
            INSERT INTO order_items
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """
        # Execute คำสั่ง SQL เพื่อเพิ่มข้อมูล
        cursor.execute(insert_query, tuple(row))
    con.commit()

In [30]:
with pyodbc.connect(conn_str) as con:
    df_query = pd.read_sql("SELECT * FROM order_items", con)
    
df_query.info()

C:\Users\Admin\AppData\Local\Temp\ipykernel_1080\3269614454.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_query = pd.read_sql("SELECT * FROM order_items", con)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB
